In [1]:
!git clone https://github.com/sk27110/DL.git

Cloning into 'DL'...
remote: Enumerating objects: 106, done.
remote: Counting objects: 100% (106/106), done.
remote: Compressing objects: 100% (90/90), done.
remote: Total 106 (delta 24), reused 85 (delta 14), pack-reused 0 (from 0)
Receiving objects: 100% (106/106), 30.67 MiB | 12.90 MiB/s, done.
Resolving deltas: 100% (24/24), done.


In [2]:
%cd DL

/content/DL


In [3]:
!git switch HW5

Branch 'HW5' set up to track remote branch 'HW5' from 'origin'.
Switched to a new branch 'HW5'


In [4]:
%cd HW5

/content/DL/HW5


In [ ]:
import wandb
wandb.login(key='ключ от аккаунта wandb')

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: sashakoch27 (sashakoch27-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [8]:
from src.dataset import get_datasets

In [ ]:
_ , _, test = get_datasets()

Using Colab cache for faster access to the 'flickr8k' dataset.
Path to dataset files: /kaggle/input/flickr8k


In [11]:
run = wandb.init()
artifact = run.use_artifact('sashakoch27-/transformer_image_captioning/my_model:v176', type='model')
artifact_dir = artifact.download()

wandb: Downloading large artifact 'my_model:v176', 190.42MB. 1 files...
wandb:   1 of 1 files downloaded.  
Done. 00:00:14.1 (13.5MB/s)


In [12]:
import wandb
run = wandb.init()
artifact = run.use_artifact('sashakoch27-/transformer_image_captioning/tokenizer:v0', type='tokenizer')
artifact_dir = artifact.download()

wandb:   1 of 1 files downloaded.  


In [13]:
from src.model import EncoderDecoder
import torch

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
checkpoint = torch.load("./artifacts/my_model:v176/best_model.pth", map_location=device)
model = EncoderDecoder(**checkpoint["model_args"]).to(device)
model.load_state_dict(checkpoint["model_state_dict"])
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 170MB/s]


EncoderDecoder(
  (encoder): Encoder(
    (cnn): Sequential(
      (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
      (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (4): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu): ReLU(inplace=True)
       

In [15]:
from src.dataset import Vocabulary

In [ ]:
tokenizer_state = torch.load("./artifacts/tokenizer:v0/tokenizer.pth", map_location=device)

In [ ]:
itos = tokenizer_state['itos']
stoi = tokenizer_state['stoi']

In [24]:
from tqdm import tqdm
from pycocoevalcap.cider.cider import Cider

In [43]:
pred_caption = dict()
true_captions = dict()

for img, captions, img_name in tqdm(test):
    input = img.unsqueeze(0).to(device)
    gen_ids = model.generate(input)[0]
    gen_tokens = [itos[idx] for idx in gen_ids if idx != stoi["<PAD>"]]
    clear_tokens = []
    for token in gen_tokens:
        if token == '<START>' or token=='<END>':
          continue
        clear_tokens.append(token)

    pred_caption[img_name] = [" ".join(clear_tokens)]
    true_captions[img_name] = captions


100%|██████████| 810/810 [00:45<00:00, 17.65it/s]


In [ ]:
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.meteor.meteor import Meteor
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider

scorers = [
    (Bleu(4), ["BLEU-1", "BLEU-2", "BLEU-3", "BLEU-4"]),
    (Meteor(), "METEOR"),
    (Rouge(), "ROUGE-L"),
    (Cider(), "CIDEr"),
]

for scorer, name in scorers:
    score, scores = scorer.compute_score(true_captions, pred_caption)

    if isinstance(name, list):
        print(f"{name[3]}: {score[3]:.3f}")
    else:
        print(f"{name}: {score:.3f}")

{'testlen': 9060, 'reflen': 8663, 'guess': [9060, 8250, 7440, 6630], 'correct': [5435, 2196, 904, 349]}
ratio: 1.0458270806878627
BLEU-4: 0.179
METEOR: 0.217
ROUGE-L: 0.456
CIDEr: 0.534
